In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import random

In [2]:
pattern = os.path.join('tr', "*.txt")
spam_files = glob.glob(pattern)
spam_counts = {}

pattern = os.path.join('ham', "*.txt")
ham_files = glob.glob(pattern)
ham_counts = {}

def get_counts(counts, files, sample_size=256):
    ix = torch.randint(0, len(files), (256,)).tolist()
    for x in ix:
        fp = files[x]
        f = open(fp, 'r')
        lines = f.read().split('\n')[5:]
        for line in lines:
            words = line.split(' ')
            words = [w.replace('\t', '').replace('!', '').replace('.', '').replace(',', '').replace('?', '').replace('*', '').lower() for w in words]
            for w in words:
                if counts.get(w): counts[w] += 1
                else: counts[w] = 1
                    
get_counts(spam_counts, spam_files)
get_counts(ham_counts, ham_files, sample_size=1024)

In [3]:
ss = sum(spam_counts.values())
print(ss)

# WHITELIST
spam_counts[' '] = 1
spam_counts[''] = 1
spam_counts['\u2007͏'] = 1
spam_counts['\u200c'] = 1

remove = []
for (k,v) in spam_counts.items():
    if v/ss > 0.02 or v/ss <= 3/ss: remove.append(k)
for r in remove: spam_counts.pop(r)



sorted_spam_counts = sorted(spam_counts.items(), key=lambda x: x[1], reverse=True)
print(sorted_spam_counts[:10])

104700
[('to', 1900), ('the', 1824), ('and', 1808), ('of', 1305), ('͏', 1273), ('you', 915), ('for', 864), ('a', 826), ('in', 794), ('on', 768)]


In [4]:
sh = sum(ham_counts.values())
print(sh)

remove = []
for (k,v) in ham_counts.items():
    if v/sh > 0.02 or v/sh <= 3/ss: remove.append(k)
for r in remove: ham_counts.pop(r)

sorted_ham_counts = sorted(ham_counts.items(), key=lambda x: x[1], reverse=True)
print(sorted_ham_counts[:10])

162711
[('and', 2556), ('the', 2281), ('\u200c', 2248), ('to', 1847), ('a', 1550), ('for', 1500), ('off', 1490), ('with', 1365), ('of', 1323), ('your', 1186)]


In [5]:
def spam_prob(w):
    ss = sum(spam_counts.values())
    sh = sum(ham_counts.values())
    ps = 0
    if spam_counts.get(w): ps = spam_counts[w]
    ph = 0
    if ham_counts.get(w): ph = ham_counts[w]
    p = (ps / ss) / ((ps / ss) + (ph / sh))
    return p

In [6]:
spam_words = []
for item in sorted_spam_counts:
    if spam_prob(item[0]) > 0.90:
        spam_words.append(item[0])
        
for w in spam_words[:5]: print(w)
print(len(spam_words))

͏
university
sponsored
admissions
best
1157


In [7]:
not_spam = []
for item in sorted_spam_counts:
    if spam_prob(item[0]) <= 0.90:
        not_spam.append(item[0])
        
for w in not_spam[:5]: print(w)
print(len(not_spam))

to
the
and
of
you
1151
